In [1]:
import sys

In [2]:
sys.path.append('/Users/deacuna/Documents/workspace/pyspark_pipes')

In [3]:
import pyspark_pipes

In [4]:
pyspark_pipes.patch()

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.getOrCreate()

In [7]:
from pyspark.ml import feature
from pyspark.sql import Row
from pyspark.ml import classification
from pyspark.ml import Pipeline

In [8]:
sc = spark.sparkContext

In [9]:
!wget 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'

--2018-04-09 18:48:32--  https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv
Resolving raw.githubusercontent.com... 151.101.116.133
Connecting to raw.githubusercontent.com|151.101.116.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60302 (59K) [text/plain]
Saving to: ‘titanic.csv.1’

titanic.csv.1       100%[===================>]  58.89K  --.-KB/s    in 0.01s   

2018-04-09 18:48:32 (3.96 MB/s) - ‘titanic.csv.1’ saved [60302/60302]



In [10]:
titanic_df = spark.read.csv('titanic.csv', header=True, inferSchema=True)

In [11]:
titanic_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



**Process title**

In [12]:
title_list_reg="|".join(['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
                    'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
                    'Don', 'Jonkheer']).lower()

In [13]:
title = feature.RegexTokenizer(pattern="\\b(" + title_list_reg + ")\\b", gaps=False, inputCol='Name') | feature.CountVectorizer()

In [14]:
gender = feature.StringIndexer(inputCol='Sex')

In [24]:
gender.fit(titanic_df)

StringIndexer_47f3b8f35cbdb7783496

In [23]:
titanic_df.select('Sex').show(5)

+------+
|   Sex|
+------+
|  male|
|female|
|female|
|female|
|  male|
+------+
only showing top 5 rows



In [15]:
pipe = (title, 
 gender,
 feature.StringIndexer(inputCol='Embarked') | feature.OneHotEncoder()
) | feature.VectorAssembler() | classification.LogisticRegression(labelCol='Survived')

In [16]:
features = ((title, 
 gender,
 feature.StringIndexer(inputCol='Embarked') | feature.OneHotEncoder()
) | feature.VectorAssembler()).fit(titanic_df.dropna(subset=['Name', 'Embarked', 'Survived', 'Sex']))

In [19]:
features.transform(titanic_df).show(4)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------------------------------------------+--------------------------------------------+------------------------------------------+------------------------------------------+------------------------------------------+--------------------------------------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|RegexTokenizer_478b82139c086ee88a71__output|CountVectorizer_4a258c4635cb00f9a8be__output|StringIndexer_47f3b8f35cbdb7783496__output|StringIndexer_496ea18e068a4e241bd2__output|OneHotEncoder_483d8b698a83e8f8aa1b__output|VectorAssembler_48a2b6ff1d5dcf03e7d3__output|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------------------------------------------+--------------------------------------------+-----------------------------------------

In [151]:
s = features.stages[0].stages[-1]

In [ ]:
featires

In [127]:
((title, 
 gender,
 feature.StringIndexer(inputCol='Embarked') | feature.OneHotEncoder()
) | feature.VectorAssembler() | classification.LogisticRegression(labelCol='Survived')).fit(titanic_df)

Py4JJavaError: An error occurred while calling o2277.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 528.0 failed 1 times, most recent failure: Lost task 0.0 in stage 528.0 (TID 521, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$5: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:395)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at org.apache.spark.storage.memory.MemoryStore.putIteratorAsValues(MemoryStore.scala:215)
	at org.apache.spark.storage.Bloc

In [114]:
titanic_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [105]:
pipe_model = pipe.fit(titanic_df)

Py4JJavaError: An error occurred while calling o594.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 65.0 failed 1 times, most recent failure: Lost task 0.0 in stage 65.0 (TID 235, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$5: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:395)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at org.apache.spark.storage.memory.MemoryStore.putIteratorAsValues(MemoryStore.scala:215)
	at org.apache.spark.storage.BlockMa